In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
# Given a m x n matrix  of m closing prices for n equities, returns an 1 x n 
# array of average closing prices
def compute_average(closing_prices_for_multiple_equities):
  sums = [0] * len(closing_prices_for_multiple_equities[0])
  for row in closing_prices_for_multiple_equities:
    for i in range(len(row)):
      sums[i] += row[i]

  avgs = [x/len(closing_prices_for_multiple_equities) for x in sums]
  return avgs

In [3]:
# Given an m x n matrix of m closing prices for n equities and n averages for n 
# equities, returns all of the difference of the closing prices and their averages
# essentially substracts mean to make the new mean of the data set 0 for each column
def compute_stocks_demeaned(closing_prices_matrix, average_closing_prices):
  m = len(closing_prices_matrix)
  n = len(closing_prices_matrix[0])
  demeaned = [[0 for i in range(n)] for j in range(m)]
  for i in range(len(closing_prices_matrix)):
    for j in range(len(closing_prices_matrix[i])):
      demeaned[i][j] = closing_prices_matrix[i][j] - average_closing_prices[j]

  return demeaned

In [4]:
# Given an m x n matrix of the demeaned closing prices for n equities, returns
# the associated covariance matrix calculated by demeaned transpose x demeaned.
def compute_covariance_matrix(demeaned):
    s_minus_m = np.array(demeaned)
    s_minus_m_t = np_demeaned.transpose()
    return np.matmul(s_minus_m_t, s_minus_m)/len(demeaned)

In [5]:
def estimated_portfolio_risk_based_on_stddev (asset_weights, covariance_matrix):
  Wt = np.asmatrix(asset_weights)
  W = Wt.transpose()
  return np.sqrt(Wt.dot(covariance_matrix).dot(W))

In [6]:
def expected_portfolio_return(average_closing_prices, asset_weights):
  M = np.array(average_closing_prices)
  W = np.array(asset_weights)
  return M.dot(W)

In [7]:
def sharpe_ratio(expected_return, portfolio_std_dev, risk_free_rate):
  return (expected_return - risk_free_rate)/portfolio_std_dev

In [8]:
# Given array of equities, returning dataframe with historical data dating back 
def get_closing_price_historical_data(equities, period, interval):
  data = {}
  for x in equities:
    ticker = yf.Ticker(x)
    history = ticker.history(period= period, interval= interval)['Close']
    data[x] = history
  return data

In [9]:
senbet = pd.read_excel('Senbet Portfolio.xlsx')
senbet

,$,Allocation,Shares
0,MCD,0.0211,142
1,AMZN.O,0.0246,12
2,FIVE.O,0.0213,169
3,TJX,0.0197,451
4,DAN,0.0266,1650
5,PG,0.0212,236
6,STZ,0.0146,97
7,EL,0.0204,106
8,COST.O,0.0201,86
9,ZTS,0.0270,259


In [10]:
trimmed_senbet = senbet
trimmed_senbet.columns = ['Ticker', 'Allocation', 'Shares']
trimmed_senbet['Allocation'] *= 100
print(trimmed_senbet)

                Ticker  Allocation  Shares
0                  MCD        2.11     142
1               AMZN.O        2.46      12
2               FIVE.O        2.13     169
3                  TJX        1.97     451
4   DAN                       2.66    1650
5                   PG        2.12     236
6                  STZ        1.46      97
7                   EL        2.04     106
8               COST.O        2.01      86
9                  ZTS        2.70     259
10                 SYK        2.42     150
11              ICLR.O        2.25     173
12                 BMY        2.48     593
13                 DGX        2.38     280
14              ABBV.K        2.82     394
15                 HON        2.96     206
16                 RTX        2.37     464
17                  WM        2.64     309
18              AAPL.O        2.28     282
19              MSFT.O        2.68     172
20                   V        3.31     236
21              CSCO.O        2.46     718
22         

In [11]:
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace('.O', ''), axis = 1)
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace('.K', ''), axis = 1)
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace(' ', ''), axis = 1)
trimmed_senbet

,Ticker,Allocation,Shares
0,MCD,2.11,142
1,AMZN,2.46,12
2,FIVE,2.13,169
3,TJX,1.97,451
4,DAN,2.66,1650
5,PG,2.12,236
6,STZ,1.46,97
7,EL,2.04,106
8,COST,2.01,86
9,ZTS,2.70,259


In [12]:
asset_weights = list(trimmed_senbet['Allocation']/100)
asset_weights

[0.021099999999999997,
 0.0246,
 0.0213,
 0.0197,
 0.0266,
 0.0212,
 0.0146,
 0.0204,
 0.020099999999999996,
 0.027000000000000003,
 0.0242,
 0.0225,
 0.0248,
 0.0238,
 0.0282,
 0.0296,
 0.023700000000000002,
 0.0264,
 0.0228,
 0.0268,
 0.0331,
 0.0246,
 0.0232,
 0.0284,
 0.0259,
 0.025,
 0.0202,
 0.0208,
 0.0109,
 0.0138,
 0.03,
 0.0308,
 0.0239,
 0.03,
 0.0138,
 0.0136,
 0.0162,
 0.0098,
 0.015599999999999998,
 0.012199999999999999,
 0.022,
 0.0216,
 0.022099999999999998,
 0.022399999999999996,
 0.0222]

In [13]:
historical_data = get_closing_price_historical_data(trimmed_senbet['Ticker'], '179d', '1d')

In [14]:
portfolio = pd.DataFrame(historical_data)
# historical_df =historical_df.dropna(axis = 0)
portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,NEE,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM
Date,,,,,,,,,,,,,,,,,,,,,
2020-08-03,190.952454,3111.889893,107.970001,51.694473,11.990716,129.782211,171.254669,196.950623,319.667328,154.078049,...,68.368225,770.373169,172.959732,96.413437,15.160692,95.451996,196.344421,92.547272,37.628666,31.495737
2020-08-04,195.824493,3138.830078,108.029999,53.079235,12.199857,132.253494,169.673447,198.192932,329.830444,155.204910,...,70.205559,784.522461,173.116684,98.068947,15.703879,94.956284,198.560059,92.200989,37.442238,31.713697
2020-08-05,195.726273,3205.030029,106.629997,53.338257,12.787441,131.907532,171.393021,200.001755,330.005188,157.279099,...,69.892776,785.125671,171.282410,101.153305,16.256763,95.333038,201.396088,94.377632,38.089825,32.189259
2020-08-06,199.576752,3225.000000,102.849998,53.208748,12.418956,131.185913,169.080505,198.014053,333.247253,160.878983,...,69.653862,783.167847,171.047012,100.867538,16.266462,96.621849,201.130203,94.902000,37.932835,32.169441
2020-08-07,200.971588,3167.459961,102.459999,55.241062,12.717729,132.016266,170.246658,198.759445,330.917633,158.435852,...,70.877937,782.811890,172.292725,102.759537,16.246853,100.091743,205.088791,98.107628,39.139698,32.020828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-13,231.320007,3400.000000,200.949997,68.459999,25.559999,135.110001,226.570007,303.000000,365.209991,163.470001,...,79.040001,705.770020,171.130005,145.679993,19.879999,118.720001,327.679993,145.990005,62.799999,45.509998
2021-04-14,230.309998,3333.000000,194.960007,68.610001,26.490000,135.600006,228.820007,304.390015,363.170013,163.080002,...,79.129997,694.309998,171.000000,153.100006,20.350000,118.190002,335.350006,147.419998,63.630001,45.389999
2021-04-15,231.279999,3379.090088,196.550003,69.239998,26.650000,137.240005,234.600006,305.130005,368.799988,165.449997,...,80.180000,718.849976,174.220001,152.220001,19.900000,119.860001,338.549988,148.529999,63.049999,46.259998


In [15]:
for index, x in trimmed_senbet.iterrows():
    portfolio[x.Ticker]*=x.Shares

In [16]:
portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,NEE,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM
Date,,,,,,,,,,,,,,,,,,,,,
2020-08-03,27115.248413,37342.678711,18246.930206,23314.207443,19784.681368,30628.601868,16611.702911,20876.765991,27491.390198,39906.214615,...,18596.157227,27733.434082,15566.375885,14269.188660,12932.070459,28349.242767,19634.442139,21841.156128,20582.880260,23716.290018
2020-08-04,27807.078064,37665.960938,18257.069794,23938.735020,20129.763651,31211.824646,16458.324326,21008.450806,28365.418213,40198.071762,...,19095.911987,28242.808594,15580.501556,14514.204132,13395.409091,28202.016220,19856.005859,21759.433350,20480.904106,23880.414167
2020-08-05,27793.130707,38460.360352,18020.469536,24055.553833,21099.278069,31130.177490,16625.123001,21200.186005,28380.446167,40735.286514,...,19010.835205,28264.524170,15415.416870,14970.689148,13867.019230,28313.912384,20139.608765,22273.121185,20835.134098,24238.511707
2020-08-06,28339.898743,38700.000000,17381.649742,23997.145287,20491.277075,30959.875488,16400.809021,20989.489655,28659.263794,41667.656479,...,18945.850464,28194.042480,15394.231110,14928.395691,13875.292364,28696.689171,20113.020325,22396.872101,20749.260540,24223.589241
2020-08-07,28537.965515,38009.519531,17315.739845,24913.719036,20984.252214,31155.838745,16513.925858,21068.501190,28458.916443,41034.885681,...,19278.798950,28181.228027,15506.345215,15208.411438,13858.565502,29727.247810,20508.879089,23153.400177,21409.414822,24111.683670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-13,32847.441040,40800.000000,33960.549484,30875.459587,42173.999119,31885.960144,21977.290710,32118.000000,31408.059265,42338.730316,...,21498.880249,25407.720703,15401.700439,21560.638916,16957.639284,35259.840363,32767.999268,34453.641296,34351.599583,34269.028736
2021-04-14,32704.019653,39996.000000,32948.241135,30943.110275,43708.499622,32001.601440,22195.540710,32265.341553,31232.621155,42237.720474,...,21523.359253,24995.159912,15390.000000,22658.800903,17358.550325,35102.430725,33535.000610,34791.119568,34805.610584,34178.669540
2021-04-15,32841.759827,40549.081055,33216.950516,31227.239037,43972.499371,32388.641296,22756.200592,32343.780518,31716.798950,42851.549210,...,21808.960083,25878.599121,15679.800110,22528.560181,16974.699675,35598.420181,33854.998779,35053.079712,34488.349583,34833.778736


In [17]:
# portfolio["PortValue"] = portfolio.sum(axis=1)
# portfolio

In [18]:
import statistics

In [19]:
pct_change = []
for i in range(1, len(portvalue)):
    curr = portvalue[i]
    yester = portvalue[i-1]
    delta = curr-yester
    pct_change.append(delta/curr * 100)

NameError: name 'portvalue' is not defined

In [31]:
avg_pct_change = statistics.mean(pct_change)
print(avg_pct_change)

0.14534711693706417 1.0541645918147067 0.12412399159775944


In [ ]:
stdev_pct_change = statistics.stdev(pct_change)
print(stdev_pct_change)

In [ ]:
print((avg_pct_change-.0145)/stdev_pct_change)

In [32]:
for sym in portfolio.columns:
    data = list(portfolio[sym])
    pct_change = []
    for i in range(1, len(data)):
        curr = data[i]
        yester = data[i-1]
        delta = curr-yester
        pct_change.append(delta/curr * 100)
    pct_change.insert(0, 0)
    portfolio[sym] = pct_change

portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,NEE,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-07-27,1.257146,1.515446,1.272400,-1.555296,1.493711,0.284990,-0.536646,0.565238,0.555541,1.837729,...,-0.943693,2.202917,1.062448,-1.177429,-0.244944,-1.882795,0.763478,0.480215,-1.515155,0.622953
2020-07-28,-2.552992,-1.829128,-1.013754,-0.579375,-0.553361,1.219893,0.350938,-0.124604,-0.009157,-0.643928,...,0.860596,-0.195348,1.973646,-2.040386,-2.898557,-1.743439,-0.694384,-1.450965,-0.340743,1.027608
2020-07-29,-0.015288,1.094433,2.480358,1.558935,1.861915,0.335128,1.150822,-0.901087,-0.438451,1.090860,...,0.497433,2.268838,1.289339,2.019483,-0.379497,1.463424,0.473890,2.347719,3.074177,-0.412731
2020-07-30,-0.409403,0.601264,-1.714853,-1.937990,-8.410430,2.366456,-0.431338,1.113805,-0.190878,1.019389,...,0.318767,1.021913,0.039745,-2.806470,-3.131129,0.074167,-1.528587,-2.155308,-2.366699,-1.132980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-05,1.590561,2.037046,1.772533,2.626917,0.162539,1.590325,0.286609,0.995700,1.629623,0.328265,...,1.315450,1.773295,0.568282,-8.161397,-1.494627,0.790551,-1.267233,1.020122,0.509960,1.431721
2021-04-06,1.319476,-0.090263,-0.871196,-0.177414,1.717254,0.036632,2.547611,-0.091522,-0.194383,-1.610013,...,1.005671,-0.222150,0.181285,-0.528463,-0.752940,-1.053461,1.076252,0.061578,-0.191600,-0.821681
2021-04-07,0.300932,1.694517,-1.422988,0.206550,-3.087690,0.175520,-0.578871,0.044050,-0.365095,1.254124,...,-0.193766,-0.622399,-0.588235,1.400374,0.839945,0.732023,-0.156181,0.666085,0.334181,0.464197


In [59]:
senbet_port = portfolio.to_numpy()
senbet_avg_returns = compute_average(senbet_port)
print(senbet_avg_returns)

[212.94185165320027, 3185.0927257005064, 155.2734636274796, 61.29330548760611, 18.12850397525553, 134.56656318536685, 202.8972437341786, 244.9086320759864, 350.33772405698977, 159.5514459876375, 222.89504263254517, 193.6285483930364, 60.63190372829331, 120.17859253270666, 98.07970633053912, 188.8949817678782, 66.6889207935866, 115.16821080213153, 121.29994495754136, 220.0822888592768, 205.84657364317826, 43.25879323948695, 66.42413421716104, 354.46810179982106, 126.43673139177888, 150.9436312627526, 35.50430171050173, 504.9297753019706, 24.85622824770112, 14.379217917692728, 1757.9878727577252, 270.83759780969035, 132.61240267620406, 65.45226980454429, 149.69453677385212, 74.23517446677778, 724.9180836597634, 164.67803068533956, 114.01516438063297, 18.64497480445734, 106.04295638952841, 249.44612564854117, 115.55017592787077, 47.727342232645555, 37.866203340072204]


In [69]:
senbet_demeaned = compute_stocks_demeaned(senbet_port, senbet_avg_returns)
# print(senbet_demeaned)

In [70]:
senbet_covmat = compute_covariance_matrix(senbet_demeaned)
print(senbet_covmat)

[[ 6.43382203e+01  1.12705867e+02  8.20977009e+01 ...  4.38268946e+01
   1.84508749e+01  1.05772512e+01]
 [ 1.12705867e+02  1.25401595e+04 -2.71926627e+02 ... -2.81798374e+02
  -1.29546836e+02 -3.73158053e+01]
 [ 8.20977009e+01 -2.71926627e+02  9.44032306e+02 ...  4.71732465e+02
   2.58459024e+02  1.23202334e+02]
 ...
 [ 4.38268946e+01 -2.81798374e+02  4.71732465e+02 ...  2.83003850e+02
   1.45679865e+02  7.13748462e+01]
 [ 1.84508749e+01 -1.29546836e+02  2.58459024e+02 ...  1.45679865e+02
   7.90926493e+01  3.70697832e+01]
 [ 1.05772512e+01 -3.73158053e+01  1.23202334e+02 ...  7.13748462e+01
   3.70697832e+01  2.06265805e+01]]


In [78]:
senbet_asset_weights = list(trimmed_senbet['Allocation']/100)

In [79]:
est_risk = estimated_portfolio_risk_based_on_stddev(senbet_asset_weights, senbet_covmat)
print(est_risk)

[[16.75346446]]


In [80]:
exp_ret = expected_portfolio_return(asset_weights=senbet_asset_weights, average_closing_prices = senbet_avg_returns)
print(exp_ret)

280.82169842356836


In [81]:
sharpe_ratio(exp_ret,est_risk, 6)

matrix([[16.40387271]])